In [10]:
from pathlib import Path
import shutil
import re

data_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID")
exported_path = data_path / "exported_large"
if not exported_path.exists():
    exported_path.mkdir()

max_person_id = 0
for subset in ["train", "test", "val"]:
    max_subset_person_id = 0
    # iterate over directories in data path
    for person_id in (data_path / f"tracklet_{subset}").iterdir():
        # iterate over files in person directory
        for img in person_id.iterdir():
            im_split = img.name.split("_")
            camera_id = int(im_split[0])
            sequence_n = int(im_split[1])
            if sequence_n % 10 != 0:
                continue

            daytime = re.search(r"[a-z]+", im_split[-1])
            if daytime:
                daytime = daytime.group(0)
                match daytime:
                    case "noon":
                        sequence_n += 1000
                    case "afternoon":
                        sequence_n += 2000
            
            person_id_int = int(person_id.name) + max_person_id
            if person_id_int > max_subset_person_id:
                max_subset_person_id = person_id_int
            
            # save the file in 0001_c1_0000.jpg format in the exported directory
            new_name = f"{person_id_int:0>4d}_c{camera_id}_{sequence_n:0>4d}.jpg"
            # copy file to exported_test/person_id directory
            shutil.copy(img, exported_path / new_name)
    max_person_id = max_subset_person_id

In [13]:
data_path = Path("/home/aleksandernagaj/Milestone/data/lsvid_large_prw_market")
data_test_dir = data_path / "test"
data_test_dir.mkdir(exist_ok=True)

for gallery_dir, query_dir in zip((data_path / "gallery").iterdir(), (data_path / "query").iterdir()):
    if not (data_test_dir / gallery_dir.name).exists():
        (data_test_dir / gallery_dir.name).mkdir()
    if not (data_test_dir / query_dir.name).exists():
        (data_test_dir / query_dir.name).mkdir()
    
    # move gallery and query images to corresponding test directory
    for img in gallery_dir.iterdir():
        shutil.move(img, data_test_dir / gallery_dir.name / img.name)
    for img in query_dir.iterdir():
        shutil.move(img, data_test_dir / query_dir.name / img.name)

In [15]:
for test_dir in data_test_dir.iterdir():
    if (data_path / "train" / data_test_dir.name).exists():
        print(f"Directory {data_test_dir.name} already exists in train directory")
        continue

    shutil.copytree(test_dir, data_path / "train" / test_dir.name)

In [14]:
data_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID/exported_train")
concat_path = Path("/home/aleksandernagaj/Milestone/data/LS-VID/concat")

for im in data_path.glob("**/*.jpg"):
    shutil.copy(im, concat_path / im.name)

In [8]:
people = img_by_person_id(data_path / "query", "*.jpeg")

for person_id, camera_id in people.items():
    # create a directory for each person id with zero padding to 4 digits
    person_dir = data_path / "query" / f"{person_id:04d}"
    person_dir.mkdir(exist_ok=True)
    # copy all images to the new directory, keeping the original files
    for _, images in camera_id.items():
        for image in images:
            shutil.copy(image, person_dir / image.name)
 